In [10]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime

In [11]:
data = {
    "keywords": "jokowi",
    "since_time": "2023-10-01",
    "until_time": "2023-10-16"
}

In [12]:
def cek_totalPagination(keywords, since_time, until_time):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    since_time = datetime.strptime(since_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    until_time = datetime.strptime(until_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page=1"
    
    response = requests.get(url,headers=headers) 
    soup = BeautifulSoup(response.text, "html.parser")
    cek = soup.find('span', {"class": "fl text"})
    hasil_text = cek.get_text()

    # Use regular expression to extract the number
    result = re.search(r'\d+', hasil_text)
    if result:
        hasil_number = int(result.group(0))
        hasil_divided = hasil_number / 9  # Divide the number by 9
        rounded_result = round(hasil_divided)  # Round to the nearest whole number
        return rounded_result
    else:
        return 0  # Return 0 if no number was found


In [13]:
jumlah_index = cek_totalPagination(data["keywords"], data["since_time"], data["until_time"])
print(f"Total results: {jumlah_index}")

Total results: 142


In [14]:
def scrape_links(page_number,keywords, since_time, until_time):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    since_time = datetime.strptime(since_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    until_time = datetime.strptime(until_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page={page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [15]:
links = []
threads_link = []
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,data["keywords"], data["since_time"], data["until_time"])))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 9 links from page 1
Scraped 9 links from page 2
Scraped 9 links from page 5
Scraped 9 links from page 3
Scraped 9 links from page 10
Scraped 9 links from page 7
Scraped 9 links from page 14
Scraped 9 links from page 15
Scraped 9 links from page 8
Scraped 9 links from page 9
Scraped 9 links from page 19
Scraped 9 links from page 12
Scraped 9 links from page 23
Scraped 9 links from page 4
Scraped 9 links from page 18
Scraped 9 links from page 17
Scraped 9 links from page 25
Scraped 9 links from page 13
Scraped 9 links from page 11
Scraped 9 links from page 6
Scraped 9 links from page 34
Scraped 9 links from page 48
Scraped 9 links from page 29
Scraped 9 links from page 27
Scraped 9 links from page 20
Scraped 9 links from page 37
Scraped 9 links from page 36
Scraped 9 links from page 35
Scraped 9 links from page 28
Scraped 9 links from page 46
Scraped 9 links from page 30
Scraped 9 links from page 45
Scraped 9 links from page 33
Scraped 9 links from page 31
Scraped 9 links from pa

In [16]:
results = []

In [17]:
def scrape_url(url,keywords):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "detail__title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "detail__author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "detail__date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('div',{"class": "page__breadcrumb"})
            if category_elements:
                category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail__body"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [18]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,data["keywords"]))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Failed to retrieve data from https://news.detik.com/berita/d-6971695/syl-direncanakan-temui-jokowi-mobil-hitam-b-8055-adt-tiba-di-istana
Failed to retrieve data from https://news.detik.com/berita/d-6964638/jokowi-singgung-kasus-bts-sudah-dipikirin-untuk-area-jauh-tapi-masalah
Failed to retrieve data from https://news.detik.com/berita/d-6963124/satgas-penanggulangan-narkoba-polri-tangkap-5-tersangka-jaringan-fredy-pratama
Failed to retrieve data from https://news.detik.com/pemilu/d-6965243/tpn-ganjar-umumkan-6-wakil-ketua-baru-termasuk-andi-gani-kspsi
Failed to retrieve data from https://news.detik.com/berita/d-6978685/ada-desakan-firli-dicopot-mahfud-biar-disikapi-sendiri-oleh-kpk
Failed to retrieve data from https://news.detik.com/pemilu/d-6960751/beri-selamat-ke-jokowi-cak-imin-mau-bikin-kereta-cepat-jakarta-surabaya
Failed to retrieve data from https://news.detik.com/pemilu/d-6976754/kaesang-kalau-mas-gibran-urusannya-tanggal-16-baru-kita-tahu
Failed to retrieve data from https://ne